In [1]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_datasets
from transformers import *
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow import keras

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm.notebook import tqdm
# from wandb.keras import WandbCallback
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
# !wget https://raw.githubusercontent.com/wangz10/contrastive_loss/master/losses.py
import losses
import feed
from feed import *
import time
import model
from model import *

2.2.0


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


2.2.0


In [2]:
train_df = pd.read_csv('files/train.csv')
val_df = pd.read_csv('files/val.csv')
test_df = pd.read_csv('files/test.csv')
pred_df = pd.read_csv('files/pred.csv')
Num_class = len(set(train_df.label.value_counts()))
print(Num_class)

6


In [3]:
BATCH_SIZE = 32
MAX_SEQ_LEN = 128
REPEAT = 2
NUM_TRAIN_ITERATION = REPEAT * int(len(train_df) / BATCH_SIZE)
NUM_TEST_ITERATION = int(len(test_df) / BATCH_SIZE)
print(NUM_TRAIN_ITERATION)
print(NUM_TEST_ITERATION)

624
31


In [4]:
import tensorflow as tf
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
# model = TFAlbertModel.from_pretrained('albert-base-v2')


In [5]:
# Reference: https://github.com/wangz10/contrastive_loss/blob/master/model.py
class UnitNormLayer(tf.keras.layers.Layer):
    '''Normalize vectors (euclidean norm) in batch to unit hypersphere.
    '''
    def __init__(self):
        super(UnitNormLayer, self).__init__()

    def call(self, input_tensor):
        norm = tf.norm(input_tensor, axis=1)
        return input_tensor / tf.reshape(norm, [-1, 1])

In [6]:
# Encoder Network
# max_seq_length = MAX_SEQ_LEN
# BATCH_SIZE = 32
def encoder_net():
#     input_ids = keras.layers.Input(shape=(max_seq_length,), name="input_ids")
#     attention_mask = keras.layers.Input(shape=(max_seq_length,), name="input_masks")
#     token_type_ids = keras.layers.Input(shape=(max_seq_length,), name="segment_ids")
    input_ids = tf.keras.Input(shape=(MAX_SEQ_LEN,), batch_size=BATCH_SIZE, name="input_ids", dtype=tf.int32)
    attention_mask = tf.keras.Input(shape=(MAX_SEQ_LEN,), batch_size=BATCH_SIZE, name="attention_mask", dtype=tf.int32)
    token_type_ids = tf.keras.Input(shape=(MAX_SEQ_LEN,), batch_size=BATCH_SIZE, name="token_type_ids", dtype=tf.int32)

    bert_input = [input_ids, attention_mask, token_type_ids]
    
#     input_ids = tf.constant(tokenizer.encode("Hello, my dog is cute"))[None, :]

    encoder = TFAlbertModel.from_pretrained('albert-base-v2')
    encoder.trainable = True

    embeddings = encoder(bert_input)
    encoder_network = tf.keras.Model(inputs=bert_input, outputs=embeddings)

    return encoder_network

# Projector Network
def projector_net():
    projector = tf.keras.models.Sequential([
        Dense(128, activation="relu"),
#         UnitNormLayer()
    ])

    return projector

In [7]:
# encoder_r = encoder_net()
# projector_z = projector_net()

In [8]:
# Convert data to InputExample format

train_examples = convert_text_to_examples(train_df['text'], train_df['label_id'])
val_examples = convert_text_to_examples(val_df['text'], val_df['label_id'])
test_examples = convert_text_to_examples(test_df['text'], test_df['label_id'])
pred_examples = convert_text_to_examples(pred_df['text'], np.zeros(len(pred_df) ))


In [9]:
# Convert to features
# max_seq_length = 128
(train_input_ids, train_input_masks, train_segment_ids, train_labels ) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=MAX_SEQ_LEN)
(val_input_ids, val_input_masks, val_segment_ids, val_labels ) = convert_examples_to_features(tokenizer, val_examples, max_seq_length=MAX_SEQ_LEN)
(test_input_ids, test_input_masks, test_segment_ids, test_labels ) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=MAX_SEQ_LEN)
(pred_input_ids, pred_input_masks, pred_segment_ids, pred_labels ) = convert_examples_to_features(tokenizer, pred_examples, max_seq_length=MAX_SEQ_LEN)

In [10]:
# BATCH_SIZE=32
train_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": tf.constant( train_input_ids, dtype=tf.int32),
                                                  "attention_mask": tf.constant( train_input_masks, dtype=tf.int32),
                                                  "token_type_ids": tf.constant( train_segment_ids, dtype=tf.int32),}
                                                    ,tf.constant(train_labels))).repeat()
train_dataset = train_dataset.shuffle(100, reshuffle_each_iteration=True).batch(BATCH_SIZE,drop_remainder=True).repeat(2)

val_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": tf.constant( val_input_ids, dtype=tf.int32),
                                                  "attention_mask": tf.constant( val_input_masks, dtype=tf.int32),
                                                  "token_type_ids": tf.constant( val_segment_ids, dtype=tf.int32),}
                                                    ,tf.constant(val_labels)))
val_dataset = val_dataset.batch(BATCH_SIZE,drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": tf.constant( test_input_ids, dtype=tf.int32),
                                                  "attention_mask": tf.constant( test_input_masks, dtype=tf.int32),
                                                  "token_type_ids": tf.constant( test_segment_ids, dtype=tf.int32),}
                                                    ,tf.constant(test_labels)))
test_dataset = test_dataset.batch(BATCH_SIZE,drop_remainder=True)

In [11]:
al_inputs,  label = next(iter(train_dataset))
print(al_inputs['input_ids'].shape)

(32, 128)


In [12]:
# print(al_inputs)

In [13]:
# print(label)
# print(al_inputs)

In [14]:
optimizer = tf.keras.optimizers.Adam()
encoder_r = encoder_net()
projector_z = projector_net()
normalization_layer = UnitNormLayer()

Some weights of the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [15]:
# r = encoder_r(al_inputs)
# # print(r)
# r1 = r[0][:,0,:]
# print(r1.shape)
# z = projector_z(r1)
# print(tf.shape(r1))
# print(tf.shape(z))

In [16]:
# @tf.function
# def train_step(bert_input, labels):
#     with tf.GradientTape() as tape:
#         r = encoder_r(bert_input, training=True)
#         r = r[0][:,0,:]
#         z = projector_z(r, training=True)
#         loss = losses.max_margin_contrastive_loss(z, labels, metric='cosine')

#     gradients = tape.gradient(loss, 
#         encoder_r.trainable_variables + projector_z.trainable_variables)
#     optimizer.apply_gradients(zip(gradients, 
#         encoder_r.trainable_variables + projector_z.trainable_variables))

#     return loss

In [17]:
# # wandb.init(project="scl", entity="authors", id="supervised-contrastive-crctd")
# EPOCHS = 5
# LOG_EVERY = 10
# train_loss_results = []

# test_count = 0

# start = time.time()
# for epoch in tqdm(range(EPOCHS)):
#     epoch_loss_avg = tf.keras.metrics.Mean()

#     for (bert_input, labels) in train_dataset:
# #         if test_count < 2 :
# #             print(bert_input)
# #             print(labels)
# #             test_count += 1
#         loss = train_step(bert_input, labels)
#         epoch_loss_avg.update_state(loss) 

#     train_loss_results.append(epoch_loss_avg.result())
# #     wandb.log({"supervised_contrastive_loss": epoch_loss_avg.result()})

#     if epoch % LOG_EVERY == 2:
#         print("Epoch: {} Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))

# end = time.time()
# print('training duration is ', end - start)
# # wandb.log({"training_time": end - start})

# with plt.xkcd():
#     plt.plot(train_loss_results)
#     plt.title("Supervised Contrastive Loss")
#     plt.show()

In [18]:
def supervised_model():
    input_ids = tf.keras.Input(shape=(MAX_SEQ_LEN,), batch_size=BATCH_SIZE, name="input_ids", dtype=tf.int32)
    attention_mask = tf.keras.Input(shape=(MAX_SEQ_LEN,), batch_size=BATCH_SIZE, name="attention_mask", dtype=tf.int32)
    token_type_ids = tf.keras.Input(shape=(MAX_SEQ_LEN,), batch_size=BATCH_SIZE, name="token_type_ids", dtype=tf.int32)

    bert_input = [input_ids, attention_mask, token_type_ids]
#     encoder_r.trainable = False
#     r = encoder_r(bert_input, training=False)

    encoder_r.trainable = True
    r = encoder_r(bert_input, training=True)
    r = r[0][:,0,:]
#     r = tf.keras.layers.GlobalAveragePooling1D()(r[0])
    r = normalization_layer(r)
    outputs = Dense(Num_class)(r)

    supervised_model = tf.keras.Model(bert_input, outputs)

    return supervised_model

In [19]:
supervised_classifier = supervised_model()

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
# metric = tf.keras.metrics.CategoricalAccuracy('accuracy')

supervised_classifier.compile(optimizer=optimizer,
    loss=loss,
    metrics=metric)

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2,
    restore_best_weights=True, verbose=2)

supervised_classifier.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(32, 128)]          0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(32, 128)]          0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(32, 128)]          0                                            
__________________________________________________________________________________________________
model (Model)                   ((32, 128, 768), (32 11683584    input_ids[0][0]                  
                                                                 attention_mask[0][0]       

In [20]:
supervised_classifier.fit(train_dataset, steps_per_epoch= NUM_TRAIN_ITERATION,
    validation_data=val_dataset,
    epochs=2,
    callbacks= es)

Epoch 1/2
624/624 [==============================] - 3358s 5s/step - loss: 0.8747 - accuracy: 0.7856 - val_loss: 0.6858 - val_accuracy: 0.8246
Epoch 2/2
624/624 [==============================] - 3340s 5s/step - loss: 0.5700 - accuracy: 0.8621 - val_loss: 0.5696 - val_accuracy: 0.8367


In [21]:
supervised_classifier.evaluate(test_dataset, steps = NUM_TEST_ITERATION)

31/31 [==============================] - 58s 2s/step - loss: 0.6320 - accuracy: 0.8034


[0.6319806575775146, 0.8034273982048035]